<a href="https://colab.research.google.com/github/elhamod/IS883/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LangChain Frameworks

##Q0: Prepation code.

Installing necessary packages.

In [1]:
!pip install langchain
!pip install pypdf
!pip install openai
!pip install tiktoken
!pip install faiss-cpu
!pip install nltk
!pip install pandas

import nltk
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 req

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Retrieve OpenAI API key.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
config_ini_location = '/content/drive/MyDrive/Synthesizing Digital Efforts/Assignment 3/config.ini' # Change this to point to the location of your config.ini file.

import configparser

config = configparser.ConfigParser()
config.read(config_ini_location)
openai_api_key = config['OpenAI']['API_KEY']

For this assignment you will use ``model_name="gpt-3.5-turbo-0613"`` only. **You are NOT allowed to use any other model. You will lose 1 point per question if you violate this requirement.**

In [14]:
model_name="gpt-3.5-turbo-0613" # Do Not change this!

**For debugging purposes for all the questions below, remember that using `verbose`  and `langchain.debug` to print the actual requests and responses is quite useful.**

## Q1:  Question Answering System Using the School's Syllabus Database (4.5 points)

 At your school, the department has embarked on a project to utilize language modeling for the development of a question-answering agent. This initiative aims to streamline the access to information for faculty and staff, particularly regarding the extensive array of courses offered at our institution. The data pertaining to these courses is currently dispersed across numerous documents within [the department's syllabus corpus](https://drive.google.com/drive/folders/1dH-t_Ujih4lMMzUOaNOHngvOYLK_gWOp?usp=sharing).

Download the corpus to your Google Drive and update the path below.

Note: The used syllabus corpus is a subset of [Cal Poly's Syllabus Corpus dataset](https://www.kaggle.com/datasets/mfekadu/syllabus-corpus).

In [ ]:
syllabus_corpus_path = "/content/drive/MyDrive/Synthesizing Digital Efforts/Assignment 4/syllabus_dataset"

First, you will use a [PyPDFDirectoryLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.pdf.PyPDFDirectoryLoader.html) to create a loader that can load all the PDFs in the directory so they could be used by LangChain.

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader(syllabus_corpus_path)
docs = loader.load()

In [ ]:
print(docs)

[Document(page_content='Eric\tOlsen\t\tCal\tPoly\t–\tIndustrial\tTechnology\tand\tPackaging\t\n1\tof\t5\t\tSyllabus\tSummer\tOnline\tGB-TGB\t2017\tIT303G.docx\tJun17\t\tRev\tE\tIT\t303\t–\tSummer\tOnline\tLean\tSix\tSigma\tGreen\tBelt\tClass\t(and\tTransactional\tGB\tand\t1st\tHalf\tBB)\t\tClass:\t\tOnline\tWed,\t5:10-7:00pm\tZoom\thttps://calpoly.zoom.us/j/215381714\tDr.\tEric\tOlsen\t\t\t\tProfessor\tof\tIndustrial\tand\tPackaging\tTechnology\t\t\t\tBldg\t03\tRm\t405\t\t\tOffice\tHours:\tAfter\tclass\tonline\tand\tby\tappointment\tOffice:\t\t805\t756-1754\t\t\t\tCourse\tWeb\tAddress:\t\tTBD\tEmail:\t\teolsen@calpoly.edu\t\tWebsite:\t\thttp://www.cob.calpoly.edu/faculty/eric-olsen/\tCentral\tCoast\tLean:\t\t\t\t\t\twww.cob.calpoly.edu/centralcoastlean/\t\tTA\tfor\tthis\tclass\tis\tPedrom\tMashaw\tpmashaw@calpoly.edu\t\tCOURSE\tDESCRIPTION\t4\tunits\tPrerequisite:\tSTAT\t217,\tSTAT\t218,\tSTAT\t251,\tor\tany\t300\tor\t400\tlevel\tstatistics\tcourse.\tDevelopment\tof\ta\tcomprehensive\t

Given the extensive data contained within these documents, it's impractical to include them in their entirety in our queries. Including all data at once could exceed the context window's capacity and may result in significant processing costs. To address this challenge, you will employ a methodical approach to manage the data effectively.

* Create a [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter): You will use a `RecursiveCharacterTextSplitter` to divide the documents into more manageable segments. This splitter will break down the documents into chunks.

* Configurations: **(0.25 point)**
  * Chunk Size Configuration: Set the `chunk_size` to 500 characters. This size ensures that the chunks are large enough to contain meaningful content but small enough to be processed efficiently.

  * Creating Overlapping Chunks: Set `chunk_overlap` to 50 characters. This overlap will help prevent the loss of context that might occur at the boundaries of each chunk. It ensures that no critical information is missed or misunderstood due to the chunking process.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 500
chunk_overlap = 50


text_splitter = RecursiveCharacterTextSplitter(separators=None, keep_separator=True, is_separator_regex=False, chunk_size=chunk_size, chunk_overlap=chunk_overlap)


Now, using the afortmentioned loader and splitter, perform the splitting.

In [ ]:
chunked_docs = text_splitter.split_documents(docs)
print(chunked_docs[3])
print(chunked_docs[1])

page_content='six\tsigma\ttransactional\tgreen\tbelt\tbody\tof\tknowledge\tincluding\tproblem\tdefinition,\tmeasurement,\tanalysis,\timprovement,\tand\tcontrol,\tas\twell\tas\tthe\tteam\tleadership\tskills\tnecessary\tto\tcomplete\tprojects.\t100%\tOnline.\t\tTEACHING\tAND\tONLINE\tLEARNING\tAPPROACH\t\tWe\twill\tuse\ta\t“flipped\tclassroom”\tapproach\tthat\temphasizes\tquestions\tand\tanswers\tand\treal\tworld\tguests\tin\tthe\tclassroom\tand\taccess\tto\tknowledge\tcontent\tand\ttheory\tonline.\t\tThe\tclass\tmeets\tsynchronously\tonline\tevery\tWednesday\tevening\tfrom\t5:10-7:' metadata={'source': '/content/drive/MyDrive/Synthesizing Digital Efforts/Assignment 4/syllabus_dataset/7___Syllabus-Summer-Online-GB-TGB-2017-IT303G.pdf', 'page': 0}
page_content='1\tof\t5\t\tSyllabus\tSummer\tOnline\tGB-TGB\t2017\tIT303G.docx\tJun17\t\tRev\tE\tIT\t303\t–\tSummer\tOnline\tLean\tSix\tSigma\tGreen\tBelt\tClass\t(and\tTransactional\tGB\tand\t1st\tHalf\tBB)\t\tClass:\t\tOnline\tWed,\t5:10-7:00pm

The next crucial step involves the creation of a data store, essentially a database, that will house the chunks of data you've created. The effectiveness of our question-answering system hinges on its ability to swiftly locate the relevant chunk containing the answer to any given query. To achieve this efficiency, we will employ a sophisticated indexing strategy, rather than relying on a basic brute-force search method.

* Build the Data Store with [Facebook AI Similarity Search (FAISS)](https://python.langchain.com/docs/integrations/vectorstores/faiss): Set up your data store using a [FAISS Vector store](https://python.langchain.com/docs/integrations/vectorstores/faiss). FAISS is a library developed by Facebook AI that allows for efficient similarity search and clustering of dense vectors.

* Embedding Calculation with `OpenAIEmbeddings`: For each chunk of data in your store, calculate an embedding using `OpenAIEmbeddings`. These embeddings are essentially numerical representations of your text data, which can then be compared to the embeddings of incoming queries.

* Indexing for Efficient Search: By creating embeddings for each chunk and indexing them in the FAISS Vector store, you will enable the system to quickly find the most relevant chunk in response to a query. This process involves comparing the embedding of the query with the embeddings of the chunks to identify the best match.

The combination of `FAISS` and `OpenAIEmbeddings` will significantly enhance the efficiency and accuracy of the question-answering system, allowing for rapid retrieval of information from the extensive syllabus corpus.

In [ ]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.0 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

With the data store and indexing system in place, you are now equipped to tackle the core functionality of our question-answering system: responding to queries based on the indexed database.

* Utilize the [*`similarity_search`*](https://python.langchain.com/docs/integrations/vectorstores/faiss) function to identify the chunk that is most relevant or most similar to the posed question. This function will compare the embedding of the query with those of the indexed chunks to find the best match. **(0.25 point)**

* Display Source Information: Once you have identified the most relevant answer, output additional details indicating where this chunk is located. Specifically, provide information about *the page number and the document from which this chunk was extracted*. **(0.5 point)**

To gain a deeper understanding of how similarity search operates, refer to the provided articles and references. These resources will offer a more detailed conceptual insight into the workings of similarity search algorithms and their applications in systems like ours.

[Resource 1.](https://www.pinecone.io/learn/what-is-similarity-search/)

[Resource 2.](https://python.langchain.com/docs/modules/data_connection/vectorstores/)

In [ ]:
question = "Who is the instructor of Linear Algebra III?"

In [ ]:
docs = db.similarity_search(question)
print(docs[0].page_content)

Math 406 – Linear Algebra III
Winter 2009
Course Syllabus
Instructor: Anton Kaul
Oﬃce: 25-312 (Faculty Oﬃces East)
Phone: 6-1678
email: akaul@calpoly.edu
Oﬃce Hours: Monday 2-4, Tuesday 9-10, Thursday 9-10
Course Web Page: www.calpoly.edu/ ∼akaul/teaching/Math406
Textbook
The text for the course is Friedberg, Insel, and Spence, Linear Algebra , 4th ed.
Course Description
In Math 406 we will continue our study of the fundamental concepts of Linear Algebra. Topics
include eigenvalues/eigenvectors and diagonalization, polynomials associated with linear transfor-
mations and the Cayley-Hamilton Theorem, inner product spaces, and the Jordan Canonical Form.
This course will focus on the theoretical aspects of the subject; proofs – and your ability to read,
write and understand them – will play a dominant role.
Homework
Homework contributes 40% to your ﬁnal course grade. Homework sets will be assigned at regular
intervals throughout the quarter (roughly once a week). Each assignment will be w

In [ ]:
docs_and_scores = db.similarity_search_with_score(question)
print(docs_and_scores[0])

(Document(page_content='Math 406 – Linear Algebra III\nWinter 2009\nCourse Syllabus\nInstructor: Anton Kaul\nOﬃce: 25-312 (Faculty Oﬃces East)\nPhone: 6-1678\nemail: akaul@calpoly.edu\nOﬃce Hours: Monday 2-4, Tuesday 9-10, Thursday 9-10\nCourse Web Page: www.calpoly.edu/ ∼akaul/teaching/Math406\nTextbook\nThe text for the course is Friedberg, Insel, and Spence, Linear Algebra , 4th ed.\nCourse Description\nIn Math 406 we will continue our study of the fundamental concepts of Linear Algebra. Topics\ninclude eigenvalues/eigenvectors and diagonalization, polynomials associated with linear transfor-\nmations and the Cayley-Hamilton Theorem, inner product spaces, and the Jordan Canonical Form.\nThis course will focus on the theoretical aspects of the subject; proofs – and your ability to read,\nwrite and understand them – will play a dominant role.\nHomework\nHomework contributes 40% to your ﬁnal course grade. Homework sets will be assigned at regular\nintervals throughout the quarter (roug

Next, you will delve deeper into the results to evaluate the system.

* Display the Top 5 Matches: Print the top five most relevant chunks in response to your query, *along with their respective similarity scores*. These scores quantify how closely each chunk matches your query, offering a clear metric of relevance. **(0.5 point)**



* Examine why certain chunks received higher or lower similarity scores. Analyze the content of each chunk in relation to your query to understand the basis of these scores. **(0.25 point)**

  * Discuss whether the model is effectively discerning relevant information or if it appears to be misled by certain elements. Provide suggestions for improvements.

[Resource.](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.faiss.FAISS.html)

In [ ]:

top_matches = docs_and_scores[:5]

print("Top 5 Matches:")
for i, (doc, score) in enumerate(top_matches, 1):
    print(f"{i}. Similarity Score: {score}")
    print(doc.page_content)
    print("\n")


for i, (doc, score) in enumerate(top_matches, 1):
    print(f"Analysis for Match {i}:")
    print(f"Similarity Score: {score}")
    print("\n")


Top 5 Matches:
1. Similarity Score: 0.31471771001815796
Math 406 – Linear Algebra III
Winter 2009
Course Syllabus
Instructor: Anton Kaul
Oﬃce: 25-312 (Faculty Oﬃces East)
Phone: 6-1678
email: akaul@calpoly.edu
Oﬃce Hours: Monday 2-4, Tuesday 9-10, Thursday 9-10
Course Web Page: www.calpoly.edu/ ∼akaul/teaching/Math406
Textbook
The text for the course is Friedberg, Insel, and Spence, Linear Algebra , 4th ed.
Course Description
In Math 406 we will continue our study of the fundamental concepts of Linear Algebra. Topics
include eigenvalues/eigenvectors and diagonalization, polynomials associated with linear transfor-
mations and the Cayley-Hamilton Theorem, inner product spaces, and the Jordan Canonical Form.
This course will focus on the theoretical aspects of the subject; proofs – and your ability to read,
write and understand them – will play a dominant role.
Homework
Homework contributes 40% to your ﬁnal course grade. Homework sets will be assigned at regular
intervals throughout the 

**Why certain chunks received higher or lower similarity scores**

* Match 1 (Score: 0.3147):

Possible Issues:
The content might not closely match the query.
There could be irrelevant information in the chunk.

* Match 2 (Score: 0.3359):

Possible Strengths:
Some degree of relevance to the query is present.
Possible Issues:
The relevance might be moderate, but not highly aligned with the query.

* Match 3 (Score: 0.3718):

Possible Strengths:
Improved relevance compared to Matches 1 and 2.
Possible Issues:
May still lack some key information relevant to the query.

* Match 4 (Score: 0.4007):

Possible Strengths:
Higher relevance compared to other matches.
May contain information closely related to the query.

**The most curious thing is the result with the highest match score has nothing to do with Linear Algebra III and only provides me with 4 matches even if I increase the number of results returned**

**Some of the suggestions for improvement are-**

  * Query Refinement: Making sure the query is clear, specific, and unambiguous. Experimenting with different phrasings to capture diverse relevant information.

  * Document Quality:
Reviewing the quality of the documents in the corpus.
Removing noisy or irrelevant documents that might be affecting scores.

  * Fine-Tuning Parameters: Experimenting with the parameters of the similarity search or text splitter. Adjusting parameters like chunk size, overlap, or similarity threshold may enhance results.






Finally, we are going to use OpenAI API to get the answer to the question based on the relevant chunk. To do that, we will LangChain's *load_qa_chain*. This [article](https://cloudatlas.me/query-your-pdfs-with-openai-langchain-and-faiss-7e8221791c62) should give you an example of how to use it.

The final step involves leveraging OpenAI API to obtain answers to your queries based on the top `k` most relevant chunk identified in the previous step **(0.25 point)**. For this, you will use LangChain's [`load_qa_chain`](https://cloudatlas.me/query-your-pdfs-with-openai-langchain-and-faiss-7e8221791c62) functionality.

* Utilize `load_qa_chain` to integrate OpenAI API into your question-answering system. This tool will enable you to send the selected chunk as a context to the API and retrieve a "precise" answer to your query.

* Track the requests sent and the responses received from the OpenAI API. This will give you visibility into the interaction between your system and the API. **(0.25 point)**

* Analyze the requests and responses in detail. Discuss how the API processes the chunk and formulates an answer **(0.5 point)**. Evaluate the overall performance of the system in leveraging OpenAI API for answering queries. Consider the relevance and precision of the answers, and how well the system integrates the information from the chunks to generate responses. **(0.5 point)**



In [ ]:
temperature = 0.7

In [ ]:
from langchain.chat_models import ChatOpenAI

# Create a reference to the language model
llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=temperature, model_name=model_name)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(), chain_type="stuff")


In [ ]:
docs = db.similarity_search(question,k=4)
chain.run(input_documents=docs, question=question)


' The instructor of Linear Algebra III is Anton Kaul.'

**Answer:**



It's important to analyze and compare the system's performance across various questions.



* Compare with First Question: Reflect on the system's response to the following question and compare it with the response to the first question above. Note any differences in accuracy, relevance, or clarity of the answers. **(0.5 point)**

* Analyze the causes behind these observations. Consider factors such as the nature of the question, the relevance of the chosen chunk, and how the AI model interprets different types of queries. **(0.25 point)**

* Propose Changes: Based on your observations, propose potential changes or adjustments that could improve the system's ability to retrieve more accurate or relevant answers **(0.25 point)**. Evaluate Trade-offs: Discuss the trade-offs associated with the changes you propose. **(0.25 point)**


In [ ]:
question2 = "What additional cost does Lean Six Sigma Black Belt Training require?"

In [ ]:
docs = db.similarity_search(question2,k=4)
chain.run(input_documents=docs, question=question2)

' $500 for CSU Students, $325 for CSU Students who have already completed or paid $300 for Lean Six Sigma Green Belt in IT303, and $3,100 for Professionals (non-CSU students).'

**Answer:**

**Compare with First Question Analyze the causes behind these observations**

Question 1 gave me the name of the professor and Question 2 gave me all possible additional costs for Lean six Sigma course.
While both the questions gave me the correct answer, I did find more details from question 2 perhaps because Question 2 is more open ended than question 1.

**Suggestion**- In this case it appears Query Refinement would be the best approach to remove ambiguity and get more specific answers.



##Q2: AI-Powered "I went to the beach" Showdown (4.5 points)

In this part, you will explore the realm of AI-driven creativity by setting up an engaging interaction between two chatbots. The aim is to test the extent of the AI's "perpetual" creativity in a fun and interactive setup. You will create a scenario where two chatbots complete each other's sentences in a themed showdown revolving around the phrase "I went to the beach."

In [ ]:
first_sentence = "I went to the beach with my son and daughter."

In [ ]:
temperature = 0.3

First, Create the `ChatOpenAI` object. Be sure to use the `model_name` above.

In [ ]:
from langchain.chat_models.openai import ChatOpenAI

llm = ChatOpenAI(temperature=temperature,openai_api_key=openai_api_key, model_name=model_name)

The next step is to construct the first AI participant, referred to as **AI_1**.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Create the prompt
template = """We are going to take turns building a story. The story should not be repetitive. It should keep adding new events.
Each turn, I will first give a sentance of no more than 20 words, and then you will give a sentence of a similar length. We repeat the cycle indefinitely.

{chat_history}
Human: {human_input}
AI:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)

# create the memory of the conversation
memory = ConversationBufferMemory(memory_key="chat_history")

# create the chain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

# get the first AI reponse
response1 = llm_chain.predict(human_input=first_sentence)
print("AI_1:", response1)



> Entering new LLMChain chain...
Prompt after formatting:
We are going to take turns building a story. The story should not be repetitive. It should keep adding new events.
Each turn, I will first give a sentance of no more than 20 words, and then you will give a sentence of a similar length. We repeat the cycle indefinitely.


Human: I went to the beach with my son and daughter.
AI:

> Finished chain.
AI_1: They built sandcastles while I soaked up the sun and watched seagulls soaring overhead.


Let's look at the conversation's memory from **AI_1**'s perspective. This should print the entire conversation **(0.25 point)**. It should look like this:

```
I went to the beach with my son and daughter.
We built sandcastles and played in the waves, laughing and splashing each other <<or something similar...>>
```

After successfully setting up **AI_1**, the next step is to develop the second participant in the creative exchange, **AI_2**. This AI will have roles that are opposite to those of **AI_1** (i.e., "Human" here is "AI" there and vice versa.).

* Setting Up `ConversationBufferMemory` (memory2): Establish a new `ConversationBufferMemory`, named `memory2`, for **AI_2**. This memory will be responsible for storing the conversation as perceived and processed by **AI_2**.
* Initializing Memory with First Sentence: Incorporate an `AIMessage`: Ensure that `memory2` already contains an `AIMessage` corresponding to the first sentence of the conversation corresponding to **AI_1**'s first response. This is crucial for setting the context from **AI_2**’s perspective. Utilize the [`add_ai_message`](https://python.langchain.com/docs/modules/memory/) function to add this initial message to `memory2`.

Consult the [provided reference](https://python.langchain.com/docs/modules/memory/conversational_customization) for a thorough understanding of message manipulation and conversation customization.

In [ ]:
# create the memory of the conversation
memory2 = ConversationBufferMemory(memory_key="chat_history")

After setting up the foundational elements for **AI_2**, your next task is to build its conversational chain, using `LLMChain` and use it to generate the next sentence in the "I went to the beach" showdown. **(0.5 point)**

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Create the prompt
template = """We are going to take turns building a story. The story should not be repetitive. It should keep adding new events.
Each turn, I will first give a sentance of no more than 20 words, and then you will give a sentence of a similar length. We repeat the cycle indefinitely.

{chat_history}
Human: {human_input}
AI:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)


# create the chain
llm_chain2 = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory2,
)

# get the first AI reponse
response2 = llm_chain2.predict(human_input=first_sentence)
print("AI_2:", response2)



> Entering new LLMChain chain...
Prompt after formatting:
We are going to take turns building a story. The story should not be repetitive. It should keep adding new events.
Each turn, I will first give a sentance of no more than 20 words, and then you will give a sentence of a similar length. We repeat the cycle indefinitely.


Human: I went to the beach with my son and daughter.
AI:

> Finished chain.
AI_2: We built sandcastles and collected seashells while enjoying the warm sun and cool ocean breeze.


At the end of your setup, `memory2.chat_memory` should have the following messages:


Conclude the setup for **AI_2** by verifying that the content of `memory2.chat_memory` correctly reflects the conversation **(0.25 point)**. It should contain the following sequence of messages:



```
I went to the beach with my son and daughter.
<<AI_1's response>>.
<<AI_2's reponse>>.
```

In [ ]:
for message_type, content in memory2.chat_memory:
    print(content)

[HumanMessage(content='I went to the beach with my son and daughter.'), AIMessage(content='We built sandcastles and collected seashells while enjoying the warm sun and cool ocean breeze.')]


Now, let's create a loop where **AI_1** and **AI_2** engage in a series of sentence completions. You'll also be tracking the total cost of the entire conversation using the `get_openai_callback` function.



* Create a loop that runs for 10 iterations. In each iteration, **AI_1** and **AI_2** will take turns completing each other’s sentences, contributing to the ongoing conversation. **(0.5 point)**

* Within the loop, after each AI has generated its response, print the responses along with the iteration number in the following format:
```
iteration X
AI_1: <<AI_1's response>>
AI_2: <<AI_2's response>>
```
Replace X with the current iteration number, and ensure the responses are accurately captured. **(0.25 point)**

* Use the [`get_openai_callback`](https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking) to track the **total** cost for the entire conversation, as opposed to per-iteration costs. **(0.25 point)**



In [ ]:
iterations=10

In [ ]:
from langchain.callbacks import get_openai_callback
from langchain.llms import OpenAI
from timeit import default_timer as timer


iterations= 10
# Loop for 10 iterations
start = timer()
for iteration in range(1, iterations + 1):
    # AI_1's turn
    with get_openai_callback() as cb1:
        response1 = llm_chain.predict(human_input="")
    print(f"Iteration {iteration}")
    print(f"AI_1: {response1}")


    # AI_2's turn
    with get_openai_callback() as cb2:
        response2 = llm_chain2.predict(human_input=response1)
    print(f"AI_2: {response2}")

    total_cost = cb1.total_cost+ cb2.total_cost
end = timer()
total_time= end-start
# Print the total cost for the entire conversation
print(f"Total Cost (USD): ${total_cost}")
print(f"Time Taken: {total_time}")



> Entering new LLMChain chain...
Prompt after formatting:
We are going to take turns building a story. The story should not be repetitive. It should keep adding new events.
Each turn, I will first give a sentance of no more than 20 words, and then you will give a sentence of a similar length. We repeat the cycle indefinitely.

Human: I went to the beach with my son and daughter.
AI: They built sandcastles while I soaked up the sun and watched seagulls soaring overhead.
Human: 
AI: Suddenly, a strong gust of wind blew away our beach umbrella, sending us scrambling to catch it.
Human: 
AI: As we were chasing after the umbrella, we stumbled upon a mysterious message in a bottle buried in the sand.
Human: 
AI: We carefully opened the bottle and read the message, which was a treasure map leading to a hidden island.
Human: 
AI: Excitedly, we decided to embark on a thrilling adventure to find the hidden island and its treasure.
Human: 
AI: We packed our bags with supplies and set sail on a 

For optimizing the token usage and cost while preserving the essence of the conversation, you'll need to adapt your code to incorporate [`ConversationSummaryBufferMemory`](https://python.langchain.com/docs/modules/memory/types/summary_buffer). This memory type will summarize the conversation when a predefined `max_token_limit=200` is reached, thus reducing the token count sent to the language model in each iteration.

* While keeping the existing language model object (`llm`) intact, recreate the chains for **AI_1** and **AI_2** to use the new `ConversationSummaryBufferMemory`. **(0.25 point)**
* Repeat the same experiment above (i.e., conducting a 10-round conversation) using the new chains. **(0.5 point)**

For a better understanding of the different types of memory and their functionalities, refer to [the following article](https://www.pinecone.io/learn/series/langchain/langchain-conversational-memory/).



In [ ]:
max_token_limit= 200

In [ ]:
# Define AI_1 and produce its first sentence.

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

# Create the prompt
template = """We are going to take turns building a story. The story should not be repetitive. It should keep adding new events.
Each turn, I will first give a sentance of no more than 20 words, and then you will give a sentence of a similar length. We repeat the cycle indefinitely.

{chat_history}
Human: {human_input}
AI:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)

# create the memory of the conversation
memory3 = ConversationSummaryBufferMemory(llm=llm, max_token_limit=max_token_limit)

conversation_with_summary3 = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory3,
    verbose=True,
)
response3= conversation_with_summary3.predict(input=first_sentence)


# get the first AI reponse

print("AI_1:", response3)

# Define AI_2 and produce its first sentence.

# create the memory of the conversation
memory4 = ConversationSummaryBufferMemory(llm=llm, max_token_limit=max_token_limit)
conversation_with_summary4 = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory4,
    verbose=True,
)
response3= conversation_with_summary4.predict(input=first_sentence)

# get the first AI reponse

print("AI_2:", response2)





> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I went to the beach with my son and daughter.
AI:

> Finished chain.
AI_1: That sounds like a wonderful family outing! Which beach did you go to?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I went to the beach with my son and daughter.
AI:

> Finished chain.
AI_2: Farewell, and may your imagination always lead you to the endless possibilities of the world.


In [ ]:
# Get the conversation going.
from langchain.callbacks import get_openai_callback
from langchain.llms import OpenAI

import time

iterations= 10
# Loop for 10 iterations
start = time.time()
for iteration in range(1, iterations + 1):
    # AI_1's turn
    with get_openai_callback() as cb1:
        response3 = conversation_with_summary3.predict(input="")
    print(f"Iteration {iteration}")
    print(f"AI_1: {response3}")


    # AI_2's turn
    with get_openai_callback() as cb2:
        response4 = conversation_with_summary4.predict(input=response3)
    print(f"AI_2: {response4}")

    total_cost = cb1.total_cost+ cb2.total_cost
end = time.time()
total_time = end-start
# Print the total cost for the entire conversation
print(f"Total Cost (USD): ${total_cost}")
print(f"Time Taken: {total_time}")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I went to the beach with my son and daughter.
AI: That sounds like a wonderful family outing! Which beach did you go to?
Human: 
AI:

> Finished chain.
Iteration 1
AI_1: We went to Santa Monica Beach. It's a popular beach in Los Angeles, California. It has a wide sandy shore, a famous pier with a Ferris wheel, and plenty of shops and restaurants nearby. It's a great place for families to enjoy the sun, sand, and ocean.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer

Answer the following questions:

* Having run the conversational AI using both standard memory and `ConversationSummaryBufferMemory`, compare the total cost incurred with each type of memory. Which memory type results in low cost? Discuss the causes behind your findings. **(0.5 point)**

* [Measure the time](https://www.programiz.com/python-programming/examples/elapsed-time) taken to execute the conversation loop with each type of memory. Determine which memory type takes longer to execute and provide reasons for this. **(0.25 point)**

* Observe the final conversations for both types of memory. Provide evidence from the code's output showing how the summary is generated, and how the conversation is condensed once the `max_token_limit` is reached. **(0.25 point)**

* Describe the process of summary generation. Assess the effectiveness of the summary in representing the original conversation. **(0.5 point)**

* What prompt engineering framework was used in `ConversationSummaryBufferMemory`? Provide evidence. **(0.25 point)**



**Answer:**


**ConversationBufferMemory**

Total Cost (USD): $0.004335

Time Taken: 24.4


**ConversationSummaryBufferMemory**

Total Cost (USD): $0.0032385

Time Taken: 25.50774383544922




**They are both pretty comparable in terms of costs and time of execution in this specific test case**

But having run multiple rounds of iteration I have found that on some occasions one does far better than the other.

The ConversationSummaryBufferMemory condenses or summarizes the conversation once it reaches the max_token_limit. This could involve selecting key sentences or important context-bearing segments to represent the original conversation more concisely.

Summary Generation Process must include selecting key information based on the token limit and providing meaningful summary. The summary does appear to be concise and captures the most important details. Making sure to capture information like, Santa Monica Pier, California, beach, windy day, activities etc.

As there is not a lot of information available as to what kind of prompt must have been engineered for the conversation summary, my best guess is that it must make sure to get rid of redundant information and keep the words that maybe have high context value. For example, Santa Monica Pier must have a strong association with Beach which would have led the AI to believe it is an important word.






##Q3 (BONUS): Riddle Me This... **(4.5 points)**

In this question, you will get to have some fun with math riddles and explore the impact of different prompt engineering frameworks on solving them using AI.

In [5]:
riddle  = "A man left 100 dollars to be divided between his two sons Alfred and Benjamin. If one third of Alfred’s legacy was taken from one-fourth of Benjamin’s, the remainder would be 11 dollars. How much is Alfred's legacy?"

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools

**Solution 0: Zero-shot learning**.

* Create an [OpenAI](https://api.python.langchain.com/en/latest/llms/langchain.llms.openai.OpenAI.html) language model object.
* Decide on a temperature value for the model. Explain your choice.
* Use the language model to get a response to the riddle. No prompt engineering at this point.
* Note AI's answer to the riddle. Is it correct or not? Discuss your observations on how well the model responded to the riddle. **(0.25 point)**



In [11]:
temperature=0.4

In [12]:
## Create the llm
from langchain.llms import OpenAI
llm = OpenAI(temperature= temperature, openai_api_key=openai_api_key)

In [13]:
ai_response = llm.predict(riddle)

# Display AI's response
print("AI's Response:")
print(ai_response)

AI's Response:


Alfred's legacy is 66 dollars.


**Answer**:

A higher temperature (e.g., 0.8 or 1.0) makes the output more diverse and creative but potentially less focused, while a lower temperature (e.g., 0.2 or 0.5) produces more deterministic and focused responses.

The AI response does not seem to be correct in the first attempt where it says Alfred's legacy is 33 Dollars. In the second attempt, the response says Alfred's legacy is 77 Dollars which is also incorrect.

In the third attempt i changed the temperature from 0.7 to 0.3 to get a more deterministic answer, however it is again incorrect.

**Solution 1: Zero-shot learning with a calculator**.


Let's try enhancing the zero-shot learning approach by integrating a calculator tool with the language model. This setup aims to improve the accuracy and effectiveness of solving riddles, especially those involving mathematical elements.

* Using [`load_tools`](https://python.langchain.com/docs/modules/agents/tools/), load the `llm-math` tool, which will serve as a calculator. This tool can be integrated with the language model to handle any mathematical computations required in the riddle-solving process.

* Using [`initialize_agent`](https://python.langchain.com/docs/modules/agents/agent_types/openai_functions_agent#using-openaifunctionsagent), create a **zero-shot agent** with the `initialize_agent` function.



In [26]:
from langchain.agents import load_tools, initialize_agent, AgentType

tool_names = ['llm-math']
math_tool = load_tools(tool_names, llm=llm)

agent_executor = initialize_agent(
    llm=llm,
    tools=math_tool,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# Display the agent's response
response = agent_executor.invoke({"input": riddle})
print("Agent's Response:")
print(response)





> Entering new AgentExecutor chain...
 I need to figure out how much each son gets and then subtract the amount taken from Alfred's legacy.
Action: Calculator
Action Input: (100/2)*(3/4) - 11
Observation: Answer: 26.5
Thought: I now know the final answer
Final Answer: Alfred's legacy is 26.5 dollars.

> Finished chain.
Agent's Response:
{'input': "A man left 100 dollars to be divided between his two sons Alfred and Benjamin. If one third of Alfred’s legacy was taken from one-fourth of Benjamin’s, the remainder would be 11 dollars. How much is Alfred's legacy?", 'output': "Alfred's legacy is 26.5 dollars."}



Invoke the agent with the riddle.


* Show evidence that the calculator tool was utilized to obtain the response by examining the intermediate steps.  **(0.5 point)**
* Evaluate whether the answer provided by the agent is correct. **(0.25 point)**
* Reflect on whether you expected the calculator to aid in solving this riddle. Discuss if the calculator tool has actually contributed to finding the solution. Include in your dicussion an explanation of how and why (or why not). **(0.5 point)**

* What prompt engineering framework was used here? Provide evidence. **(0.25 point)**


**Answer:**

The calculator was used as the calculation (100/2)*(3/4) - 11 is correct, however the end answer is still incorrect as the model has assumed the legacy was to be divided equally between Alfred and Benjamin. Because of this assumption the model came to a wrong answer. The calculator tool performed its role in executing calculations but couldn't correct the underlying assumption made by the model. The prompt engineering here follows an explicit instruction framework. The goal was to guide the model explicitly by including detailed instructions about the distribution of the legacy, ratios, and relationships between Alfred and Benjamin's shares.



**Solution 3: Few-shot Learning**.

Now, you'll provide the model with examples to learn from. This approach aims to improve AI's ability to solve riddles by presenting it with similar problems and their solutions.

* Create three examples of riddles that are similar to the one you are given. For each example riddle, only provide a clear and correct *final* solution. This will serve as a learning reference for the AI. **(0.25 point)**
* Create a `PromptTemplate` that formats the example riddles and their solutions appropriately. Utilize the prompt to create a `FewShotPromptTemplate`. This template will be used to present the example riddles and their solutions to the AI in a way that facilitates learning. Refer to [this reference](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples) for guidance.
* Introduce slight variations in the answers to reduce *social* biases. Consider adding other variations that could enhance learning. Discuss your implementation. **(0.25 point)**

In [ ]:
riddle1= "A wealthy individual decided to distribute $1200 among their three children: Claire, David, and Emily. If Claire's share is twice the sum of David's and Emily's combined, and Emily's share is three-fifths of David's share, determine the amount each child received."
riddle2= "A grandfather left $8000 to be divided among his four grandchildren: Alice, Bob, Charlie, and David. If Alice's share is equal to the average of Bob and Charlie's shares, and David's share is half of Alice's share, calculate the amount each grandchild received."
riddle3= "A treasure chest contains 500 gold coins to be shared among three pirates: Captain Redbeard, Black-Eyed Morgan, and Silent Anne. If Redbeard's share is three times the sum of Morgan's and Anne's combined, and Anne's share is one-fourth of Morgan's share, determine the number of gold coins each pirate received."




**Answer**:

Now, call the `llm` object with the template after properly substituting the riddle into it.


* Explain how you obtained the used examples. **(0.25 point)**
* Show evidence that the template was utilized to obtain the response by examining the intermediate steps.  **(0.25 point)**
* Evaluate whether the answer provided by the framework is correct. **(0.25 point)**
* Reflect on whether you expected the framework to aid in solving this riddle. Discuss if it has actually contributed to finding the solution. Include in your dicussion an explanation of how and why (or why not). **(0.5 point)**

**Answer:**



**Solution 3: Chain of Thought (CoT)**.


Finally, you'll guide the model through a step-by-step process, breaking down the solution into clear, logical steps. This CoT approach helps the AI model understand the reasoning process needed to arrive at the correct answer.


* Create a detailed example of a similar riddle and logically break it down into steps. The steps involve translating the problem into an alogrithm with equation solving. Clearly outline each step. **(0.25 point)**
* Ensure that each step in the solution is explained clearly and simply. Avoid complex jargon or overly technical language. Make sure the steps follow a logical sequence that is easy to understand and replicate.
* Use the same `PromptTemplate` object from the previous solution to maintain consistency in formatting.
* Create a new `FewShotPromptTemplate` that incorporates the detailed example you've created. This template will guide the AI model in applying a similar chain of thought to solve other riddles.

Now, call the `llm` object with the template after properly substituting the riddle into it.


* Evaluate whether the answer provided by the framework is correct. **(0.25 point)**
* Reflect on whether you expected the framework to aid in solving this riddle. Discuss if it has actually contributed to finding the solution. Include in your dicussion an explanation of how and why (or why not). **(0.5 point)**

**Answer:**

